In [1]:
!pip install transformers
!pip install datasets

NameError: name 'datasets' is not defined

In [3]:
from datasets import load_dataset
dataset=load_dataset('AiresPucrs/toxic-comments')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/536 [00:00<?, ?B/s]

toxic_content_dataset.csv:   0%|          | 0.00/15.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/70157 [00:00<?, ? examples/s]

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment_text', 'toxic'],
        num_rows: 70157
    })
})

In [5]:
dataset["train"][0]

{'comment_text': 'explanation edits made username hardcore metallica fan reverted vandalisms closure gas voted new york dolls fac please remove template talk page since retired',
 'toxic': 1}

preprocessing task

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer=AutoTokenizer.from_pretrained("bert-base-uncased")
def tokenize_function(examples):
  return tokenizer(examples["comment_text"],truncation=True,padding="max_length")
## map applies that function in all rows
tokenized_dataset=dataset.map(tokenize_function,batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/70157 [00:00<?, ? examples/s]

In [9]:
tokenized_dataset


DatasetDict({
    train: Dataset({
        features: ['comment_text', 'toxic', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 70157
    })
})

In [22]:
dataset = tokenized_dataset.rename_column("toxic", "labels")




In [23]:
dataset

DatasetDict({
    train: Dataset({
        features: ['comment_text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 70157
    })
})

In [ ]:
tokenized_dataset

In [ ]:
tokenized_dataset["train"][0]

In [24]:
from transformers import TrainingArguments
training_args=TrainingArguments(
    output_dir='./results',
    #eval_strategy="epoch",
    learning_rate=2e-5,
    #per_device_eval_batch_size=16,
    per_device_train_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
)
training_args

TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=None,
eval_strategy=IntervalStrategy.NO,
eval_use_gather_object=False,


In [18]:
from transformers import Trainer,AutoModelForSequenceClassification

In [25]:
dataset["train"]

Dataset({
    features: ['comment_text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 70157
})

In [26]:
model=AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased",num_labels=2)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    ##eval_dataset=dataset["test"],

    ##data_collator=data_collator,
)

trainer.train()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.228100
1000,0.169200
1500,0.164500
2000,0.156300


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Step,Training Loss
500,0.228100
1000,0.169200
1500,0.164500
2000,0.156300
2500,0.153200
3000,0.138900
3500,0.138000
4000,0.140400


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


TrainOutput(global_step=4385, training_loss=0.15817418201755362, metrics={'train_runtime': 6412.4294, 'train_samples_per_second': 10.941, 'train_steps_per_second': 0.684, 'total_flos': 1.845908231089152e+16, 'train_loss': 0.15817418201755362, 'epoch': 1.0})

In [ ]:
trainer.evaluate()

In [ ]:
#save the model
model.save_pretrained("./fine-tuned-model")
tokenizer.save_pretrained("./fine-tuned-model")